'org.apache.spark:spark-sql-kafka-0-10_2.13:3.2.0'

In [4]:
from pyspark.sql import SparkSession

scala_version = '2.13'
spark_version = '3.5.1'
packages = [
    f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}',
    'org.apache.kafka:kafka-clients:3.2.0'
]

spark = SparkSession.builder\
   .master("local[*]")\
   .appName("kafka-example-1")\
   .config("spark.jars.packages", ",".join(packages))\
   .getOrCreate()

spark

In [2]:
data = ['Hello', 'World']
df = spark.createDataFrame([{'value': v} for v in data])

In [15]:
topic_name = 'test_topic'
df.write.format("kafka")\
  .option("kafka.bootstrap.servers", "localhost:9092")\
  .option("topic", topic_name)\
  .save()

Py4JJavaError: An error occurred while calling o118.save.
: java.lang.NoClassDefFoundError: Could not initialize class org.apache.spark.sql.kafka010.KafkaSourceProvider$
	at org.apache.spark.sql.kafka010.KafkaSourceProvider.createRelation(KafkaSourceProvider.scala:180)
	at org.apache.spark.sql.execution.datasources.SaveIntoDataSourceCommand.run(SaveIntoDataSourceCommand.scala:48)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:75)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult(commands.scala:73)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.executeCollect(commands.scala:84)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:107)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$6(SQLExecution.scala:125)
	at org.apache.spark.sql.execution.SQLExecution$.withSQLConfPropaga

In [16]:
from pyspark.sql.functions import col, concat, lit

kafkaDf = spark.read.format("kafka")\
  .option("kafka.bootstrap.servers", "localhost:9092")\
  .option("subscribe", topic_name)\
  .option("startingOffsets", "earliest")\
  .load()

Py4JJavaError: An error occurred while calling o124.load.
: java.lang.NoClassDefFoundError: Could not initialize class org.apache.spark.sql.kafka010.KafkaSourceProvider$
	at org.apache.spark.sql.kafka010.KafkaSourceProvider.org$apache$spark$sql$kafka010$KafkaSourceProvider$$validateBatchOptions(KafkaSourceProvider.scala:348)
	at org.apache.spark.sql.kafka010.KafkaSourceProvider.createRelation(KafkaSourceProvider.scala:128)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:346)
	at org.apache.spark.sql.DataFrameReader.loadV1Source(DataFrameReader.scala:229)
	at org.apache.spark.sql.DataFrameReader.$anonfun$load$2(DataFrameReader.scala:211)
	at scala.Option.getOrElse(Option.scala:201)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:211)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:172)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.r

In [17]:
query = df.writeStream.format("console").start()
import time
time.sleep(10) # sleep 10 seconds
query.stop()

AnalysisException: [WRITE_STREAM_NOT_ALLOWED] `writeStream` can be called only on streaming Dataset/DataFrame.

phần bên dưới dùng python bình thường chạy consumer ok nhưng pyspark phía trên ko đc => lỗi không phải do kafka

In [47]:
import json 
from kafka import KafkaConsumer

consumer = KafkaConsumer(
        'test_topic',
        bootstrap_servers='localhost:9092',
        auto_offset_reset='earliest'
    )
for message in consumer:
    print(json.loads(message.value))

{'gender': 'male', 'name': {'title': 'Mr', 'first': 'Vir', 'last': 'Dragan'}, 'location': {'street': {'number': 9501, 'name': 'Holmogorskiy provulok'}, 'city': 'Balakliya', 'state': 'Zaporizka', 'country': 'Ukraine', 'postcode': 76335, 'coordinates': {'latitude': '75.2747', 'longitude': '-94.4065'}, 'timezone': {'offset': '+7:00', 'description': 'Bangkok, Hanoi, Jakarta'}}, 'email': 'vir.dragan@example.com', 'login': {'uuid': '24dfe04b-7f92-4f9c-b7b3-5c22b674c483', 'username': 'redduck427', 'password': 'homeboy', 'salt': 'E7Jk7LV7', 'md5': '69a26b14e67acca97e2984cbd7e4f246', 'sha1': 'f83593362a0af967c7ec7e03fc3b8a30d45ffaad', 'sha256': '258c9ddd6291d61ecde5834fb1b870dd83aa2ca7233c23a141c1c67789bb0706'}, 'dob': {'date': '1989-11-21T13:20:45.539Z', 'age': 34}, 'registered': {'date': '2005-02-11T04:32:25.610Z', 'age': 19}, 'phone': '(098) L43-7252', 'cell': '(068) S83-0135', 'id': {'name': '', 'value': None}, 'picture': {'large': 'https://randomuser.me/api/portraits/men/14.jpg', 'medium':

KeyboardInterrupt: 